In [3]:
from sampo.scheduler.selection.neural_net import NeuralNet, load_dataset
import torch
import pandas

from torch import nn
from torch.autograd import Variable

input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 1
learning_rate = 0.001

model = NeuralNet(input_parameters, layer_size, classification_size)

x_train, x_test, y_train, y_test = load_dataset('dataset.csv')
model.fit(x_train, y_train, 100)

Epoch [1/100], Step [100/112], Loss: 0.2998
Epoch [2/100], Step [100/112], Loss: 0.3878
Epoch [3/100], Step [100/112], Loss: 0.4226
Epoch [4/100], Step [100/112], Loss: 0.4352
Epoch [5/100], Step [100/112], Loss: 0.4364
Epoch [6/100], Step [100/112], Loss: 0.4321
Epoch [7/100], Step [100/112], Loss: 0.4252
Epoch [8/100], Step [100/112], Loss: 0.4175
Epoch [9/100], Step [100/112], Loss: 0.4095
Epoch [10/100], Step [100/112], Loss: 0.4018
Epoch [11/100], Step [100/112], Loss: 0.3944
Epoch [12/100], Step [100/112], Loss: 0.3874
Epoch [13/100], Step [100/112], Loss: 0.3809
Epoch [14/100], Step [100/112], Loss: 0.3747
Epoch [15/100], Step [100/112], Loss: 0.3689
Epoch [16/100], Step [100/112], Loss: 0.3634
Epoch [17/100], Step [100/112], Loss: 0.3582
Epoch [18/100], Step [100/112], Loss: 0.3532
Epoch [19/100], Step [100/112], Loss: 0.3485
Epoch [20/100], Step [100/112], Loss: 0.3439
Epoch [21/100], Step [100/112], Loss: 0.3395
Epoch [22/100], Step [100/112], Loss: 0.3353
Epoch [23/100], Ste

In [5]:
from sampo.scheduler.selection.neural_net import encode_graph
from copy import deepcopy
from sampo.scheduler.multi_agency.multi_agency import Agent, Manager
from sampo.scheduler.topological.base import TopologicalScheduler
from sampo.scheduler.heft.base import HEFTScheduler, HEFTBetweenScheduler
import time
from sampo.generator.base import SimpleSynthetic
from sampo.scheduler.multi_agency.block_graph import BlockGraph, BlockNode

ss = SimpleSynthetic(256)
schedulers = [HEFTScheduler(), HEFTBetweenScheduler(), TopologicalScheduler()]
contractors = [ss.contractor(10)]

print('   NeuralNet ------ Multi-Agency')
print('Result | Time ---- Result | Time')

with torch.no_grad():
    for _ in range(100):
        wg = ss.work_graph(top_border=100)
        bg = BlockGraph([BlockNode(wg)])
        # encode graph
        encoding = torch.Tensor(encode_graph(wg))

        nn_start = time.time()
        # predict scheduler
        outputs = model(encoding)
        _, predicted = torch.max(outputs.data, 0)
        # use predicted scheduler to generate schedule
        nn_result = schedulers[predicted].schedule(wg, contractors).execution_time
        nn_time = time.time() - nn_start

        agents = [Agent(str(scheduler), scheduler, deepcopy(contractors)) for scheduler in schedulers]
        manager = Manager(agents)
        ma_start = time.time()
        # run MA scheduler selection
        block_schedule = list(manager.manage_blocks(bg).values())
        ma_result = max(v.end_time for v in block_schedule)
        ma_time = time.time() - ma_start

        if schedulers[predicted] != block_schedule[0].agent.scheduler:
            print(f'Prediction mismatch | predicted: {schedulers[predicted]}, actual: {block_schedule[0].agent.scheduler}')

        print(f'{block_schedule[0].agent.scheduler} {nn_result} | {nn_time} ---- {ma_result} | {ma_time} ------------- {int(ma_time / nn_time * 100)}% time win | {int((1 - ma_result / nn_result) * 100)}% result loss')


   NeuralNet ------ Multi-Agency
Result | Time ---- Result | Time
HEFTAddEnd 1522 | 0.13998723030090332 ---- 1523 | 0.9100446701049805 ------------- 650% time win | 0% result loss
HEFTAddEnd 1010 | 0.09212827682495117 ---- 1011 | 0.5005624294281006 ------------- 543% time win | 0% result loss
Prediction mismatch | predicted: HEFTAddEnd, actual: HEFTAddBetween
HEFTAddBetween 1208 | 0.09966230392456055 ---- 1206 | 0.57659912109375 ------------- 578% time win | 0% result loss
HEFTAddEnd 490 | 0.04187154769897461 ---- 491 | 0.15395617485046387 ------------- 367% time win | 0% result loss
Prediction mismatch | predicted: HEFTAddEnd, actual: HEFTAddBetween
HEFTAddBetween 1045 | 0.0936136245727539 ---- 1017 | 0.40436458587646484 ------------- 431% time win | 2% result loss
HEFTAddEnd 1125 | 0.1050102710723877 ---- 1126 | 0.5630557537078857 ------------- 536% time win | 0% result loss
HEFTAddEnd 1359 | 0.1416935920715332 ---- 1360 | 0.7902233600616455 ------------- 557% time win | 0% result lo